<a href="https://colab.research.google.com/github/SwapnilBhandarkar/BOOKS/blob/main/Face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install deepface pandas
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from datetime import datetime
from deepface import DeepFace
import re
import cv2

known_path = "/content/drive/MyDrive/attendance/known"
unknown_path = "/content/drive/MyDrive/attendance/unknown"
# attendance_file = "/content/drive/MyDrive/attendance.csv"
attendance_file = "/content/drive/MyDrive/attendance/attendance.csv"

expected_columns = ["Name_Enroll", "Date", "Time"]
if not os.path.exists(attendance_file):
    df = pd.DataFrame(columns=expected_columns)
    df.to_csv(attendance_file, index=False)
else:
    df = pd.read_csv(attendance_file)
    if list(df.columns) != expected_columns:
        df = pd.DataFrame(columns=expected_columns)
        df.to_csv(attendance_file, index=False)

df = pd.read_csv(attendance_file)


def has_human_face(image_path):
    # Use OpenCV's Haar cascade for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    img = cv2.imread(image_path)
    if img is None:
        return False
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    return len(faces) > 0



def remove_non_human_images(folder_path):
    removed = 0
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if not os.path.isfile(file_path):
            continue
        try:
            result = has_human_face(file_path)
            print(f"Checking {file_path}: Face detected? {result}")
            if not result:
                os.remove(file_path)
                removed += 1
                print(f"Removed non-human image: {file_path}")
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
    print(f"Total removed from {folder_path}: {removed}")

# Remove non-human images from both known and unknown folders
remove_non_human_images(known_path)
remove_non_human_images(unknown_path)

def get_datetime_from_filename(filename):
    # Example: login_20250817_213700.jpg
    match = re.search(r'login_(\d{8})_(\d{6})', filename)
    if match:
        date_part = match.group(1)  # '20250817'
        time_part = match.group(2)  # '213700'
        date_str = f"{date_part[6:8]}/{date_part[4:6]}/{date_part[0:4]}"
        time_obj = datetime.strptime(time_part, "%H%M%S")
        time_str = time_obj.strftime("%I:%M:%S %p")
        return date_str, time_str
    else:
        now = datetime.now()
        return now.strftime("%d/%m/%Y"), now.strftime("%I:%M:%S %p")

def remove_all_attendance():
    df = pd.DataFrame(columns=expected_columns)
    df.to_csv(attendance_file, index=False)
    print("All attendance records have been removed.")
def remove_all_known_images():
    known_path = "/content/drive/MyDrive/attendance/known"  # Update if your path is different
    removed = 0
    for filename in os.listdir(known_path):
        file_path = os.path.join(known_path, filename)
        if os.path.isfile(file_path):
            try:
                os.remove(file_path)
                removed += 1
            except Exception as e:
                print(f"Error removing {file_path}: {e}")
    print(f"Removed {removed} images from known folder.")


for file in os.listdir(unknown_path):
    unknown_file = f"{unknown_path}/{file}"
    if not os.path.isfile(unknown_file):
        continue

    try:
        best_match = None
        best_distance = float('inf')
        best_student = None
        for known_file in os.listdir(known_path):
            known_file_path = f"{known_path}/{known_file}"
            print(f"Comparing {file} with {known_file}...")

            result = DeepFace.verify(img1_path=unknown_file, img2_path=known_file_path, enforce_detection=False)
            # print(result)
            print(f"Match: {result['verified']}")

            if result["verified"] and result["distance"] < best_distance:
                best_match = result
                best_distance = result["distance"]
                best_student = os.path.splitext(known_file)[0]

        if best_match:
            date_str, time_str = get_datetime_from_filename(file)

            already_present = ((df["Name_Enroll"] == best_student) & (df["Date"] == date_str)).any()
            if already_present:
                  print(f"⚠️ {best_student} already in class for {date_str}!")
            else:
                df.loc[len(df)] = [best_student, date_str, time_str]
                df.to_csv(attendance_file, index=False)
                df = pd.read_csv(attendance_file)
                print(f"✅ {best_student} logged in at {date_str} {time_str}!")
            os.remove(unknown_file)
            # os.remove(known_file)

        else:
            print(f"{file} ❌ User not registered")

    except Exception as e:
        print(f"{file} ❌ Error: {e}")


Checking /content/drive/MyDrive/attendance/known/Swapnil_11.jpg: Face detected? True
Total removed from /content/drive/MyDrive/attendance/known: 0
Checking /content/drive/MyDrive/attendance/unknown/login_20250822_093541.jpg: Face detected? True
Total removed from /content/drive/MyDrive/attendance/unknown: 0
Comparing login_20250822_093541.jpg with Swapnil_11.jpg...
25-08-22 04:07:19 - 🔗 vgg_face_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5 to /root/.deepface/weights/vgg_face_weights.h5...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: /root/.deepface/weights/vgg_face_weights.h5
100%|██████████| 580M/580M [00:06<00:00, 95.1MB/s]


Match: True
✅ Swapnil_11 logged in at 22/08/2025 09:35:41 AM!


In [ ]:
# remove_all_attendance()
remove_all_known_images()

Removed 1 images from known folder.
